<a href="https://colab.research.google.com/github/imandrec/Thermal-Camera/blob/main/VIDEO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.interpolate import RectBivariateSpline
import re
import os
import shutil
from google.colab import files
import cv2

# Upload the file
uploaded = files.upload()

# Assuming the file name is 'ThermalCameraAMG.txt'
file_name = 'ThermalCameraAMG.txt'

# Verify if the file exists in the current directory
if os.path.exists(file_name):
    print("File exists")
else:
    print("File does not exist")

# Set the output directory to the "Downloads/output_images" folder
output_dir = '/content/output_images'

# Create the directory if it doesn't exist
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Function to read the data from the text file
def read_data_from_file(file_path):
    with open(file_path, 'r') as file:
        content = file.read()

    # Split the content by elapsed time
    blocks = re.split(r'Elapsed Time: (\d+)', content)[1:]

    elapsed_times = []
    images = []

    for i in range(0, len(blocks), 2):
        elapsed_time = int(blocks[i].strip())
        image_data_str = blocks[i+1].strip()

        # Remove unnecessary characters and split into rows
        rows = [row.strip('[], ') for row in image_data_str.split('\n') if row and 'Elapsed Time' not in row and '-------------' not in row]

        # Split each row into float values
        image_data = []
        for row in rows:
            try:
                image_data.append(list(map(float, row.split(','))))
            except ValueError:
                continue

        elapsed_times.append(elapsed_time)
        images.append(np.array(image_data))

    return elapsed_times, images

# Function to generate and save high-resolution images
def generate_and_save_images(elapsed_times, images, output_dir):
    for elapsed_time, low_res_image in zip(elapsed_times, images):
        x = np.arange(0, low_res_image.shape[0])
        y = np.arange(0, low_res_image.shape[1])
        interp_func = RectBivariateSpline(x, y, low_res_image)

        x_new = np.linspace(0, low_res_image.shape[0] - 1, 32)
        y_new = np.linspace(0, low_res_image.shape[1] - 1, 32)
        high_res_image = interp_func(x_new, y_new)

        # Define the color map to use for the images
        cmap = plt.cm.jet

        # Plot and save the high-resolution image
        image_path = os.path.join(output_dir, f'image_{elapsed_time}.png')
        plt.imshow(high_res_image, cmap=cmap)
        plt.title(f'Elapsed Time: {elapsed_time}')
        plt.colorbar()
        plt.savefig(image_path)
        plt.close()

# Function to create a video from images
def create_video_from_images(image_folder, video_file):
    images = [img for img in sorted(os.listdir(image_folder)) if img.endswith(".png")]
    frame = cv2.imread(os.path.join(image_folder, images[0]))
    height, width, layers = frame.shape

    video = cv2.VideoWriter(video_file, cv2.VideoWriter_fourcc(*'mp4v'), 2, (width, height))

    for image in images:
        video.write(cv2.imread(os.path.join(image_folder, image)))

    cv2.destroyAllWindows()
    video.release()

# Main code
elapsed_times, images = read_data_from_file(file_name)
generate_and_save_images(elapsed_times, images, output_dir)

# Create the video from the images
video_file = '/content/output_images/video.mp4'
create_video_from_images(output_dir, video_file)

# Download the video file to your local machine
files.download(video_file)

print(f'Images saved to {output_dir}, video created at {video_file}, and video file downloaded')


Saving ThermalCameraAMG.txt to ThermalCameraAMG (1).txt
File exists


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Images saved to /content/output_images, video created at /content/output_images/video.mp4, and video file downloaded
